In [ ]:
from google.colab import drive
import sys
import torch
from torch.utils.data import DataLoader, random_split
from torchvision.transforms import transforms
import matplotlib.pyplot as plt

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
sys.path.append('/content/drive/My Drive/Colab Notebooks/fasterRCNN/')

import utils  # Utility functions for training and evaluation
from model import get_model
from dataset import RoadDamageDataset

# Paths
dataset_path = "/content/drive/My Drive/Colab Notebooks/fasterRCNN/United_States/train"

# Configuration
num_classes = 8  # Adjust based on your dataset
batch_size = 4
learning_rate = 0.005
num_epochs = 30
validation_split = 0.2  # Use 20% of the data for validation
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Dataset and Transforms
transform = transforms.Compose([
    transforms.ToTensor()
])

dataset = RoadDamageDataset(
    root=dataset_path,
    transforms=transform
)

# Split into training and validation datasets
dataset_size = len(dataset)
val_size = int(validation_split * dataset_size)
train_size = dataset_size - val_size

train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=utils.collate_fn)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=utils.collate_fn)

# Load model
model = get_model(num_classes)
model.to(device)

# Optimizer and learning rate scheduler
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Training and validation
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    train_loss = 0.0
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        train_loss += losses.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # Validation phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, targets in val_loader:
            images = [img.to(device) for img in images]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            loss_dict = model(images, targets)
            losses = sum(loss for loss in loss_dict.values())


            val_loss += losses.item()

    val_loss /= len(val_loader)
    val_losses.append(val_loss)

    # Adjust learning rate
    lr_scheduler.step()

    print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")

# Plot training and validation losses
plt.figure(figsize=(10, 5))
plt.plot(range(1, num_epochs + 1), train_losses, label="Training Loss")
plt.plot(range(1, num_epochs + 1), val_losses, label="Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.grid()
plt.show()

# Save the trained model
torch.save(model.state_dict(), '/content/drive/MyDrive/faster_rcnn_road_damage.pth')
